In [ ]:
import os
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import cv2
import rasterio
import rioxarray as rxr
import geopandas as gpd
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
from matplotlib.colors import ListedColormap
import imutils


# NDVI Calculation with sentinel hub API

def ndvi_api(red,nir):
    
    '''
    NDVI calculation with sentinel hub API (using red & nir value arrays)
    
    red: red array
    nir: NIR array
    
    retun: ndvi 2D array
    '''
    
    # calculate the NDVI
    ndvi = es.normalized_diff(nir, red)
    
    return ndvi

# Calculate the green area

def green_area(ndvi,res,thresh=0.6):
    
    '''
    Calculate the green area with specific threshold values
    
    ndvi: ndvi array
    res: image resolution
    thresh: threshold value to distinguish forect/ green area
    
    return: green cover in km2
    '''
    
    # get the ndvi data from masked array
    ndvi_data = ma.getdata(ndvi)
    # get the row & col count
    rows, cols = ndvi_data.shape
    
    # replace ndvi values with 1, if ndvi >= threshold value
    for i in range(0,rows-1):
        for j in range(0,cols-1):
            if ndvi_data[i][j] >= thresh:
                ndvi_data[i][j] = 1
            else:
                ndvi_data[i][j] = 0
            
    # count the ones
    ndvi_list = ndvi_data.tolist()
    green_pixel = sum(x.count(1) for x in ndvi_list)
    
    # calculate the green area in km2
    green_area = (green_pixel*res*res)/1000000
    
    print('Forest/ Green area: ',green_area,"km2")
    
    return green_area

# Forest/ green cover with user input total area
def green_cover(green_area,total_area):
    green_cover = np.round((green_area/total_area)*100,2)
    print("Forest/green cover: ",green_cover,"%")